In [27]:
import pandas as pd
import math
# function to calculate the entropy of entire dataset
def base_entropy(dataset):
    p = 0
    n = 0
    target = dataset.iloc[:, -1]
    targets = list(set(target))
    for i in target:
        if i == targets[0]:
            p = p + 1
        else:
            n = n + 1
    if p == 0 or n == 0:
        return 0
    elif p == n:
        return 1
    else:
        entropy = 0 - (((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy
    
# function to calculate the entropy of attributes
def entropy(dataset, feature, attribute):
    p = 0
    n = 0
    target = dataset.iloc[:, -1]
    targets = list(set(target))
    for i, j in zip(feature, target):
        if i == attribute and j == targets[0]:
            p = p + 1
        elif i == attribute and j == targets[1]:
            n = n + 1
    if p == 0 or n == 0:
        return 0
    elif p == n:
        return 1
    else:
        entropy = 0 - (
            ((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy


# -----------------------------------------------------------------------


# a utility function for checking purity and impurity of a child
# -----------------------------------------------------------------------
def counter(target, attribute, i):
    p = 0
    n = 0
    targets = list(set(target))
    for j, k in zip(target, attribute):
        if j == targets[0] and k == i:
            p = p + 1
        elif j == targets[1] and k == i:
            n = n + 1
    return p, n


# -----------------------------------------------------------------------


# function that calculates the information gain
# -----------------------------------------------------------------------
def Information_Gain(dataset, feature):
    Distinct = list(set(feature))
    Info_Gain = 0
    for i in Distinct:
        Info_Gain = Info_Gain + feature.count(i) / len(feature) * entropy(dataset, feature, i)
    Info_Gain = base_entropy(dataset) - Info_Gain
    return Info_Gain


# -----------------------------------------------------------------------


# function that generates the childs of selected Attribute
# -----------------------------------------------------------------------
def generate_childs(dataset, attribute_index):
    distinct = list(dataset.iloc[:, attribute_index])
    childs = dict()
    for i in distinct:
        childs[i] = counter(dataset.iloc[:, -1], dataset.iloc[:, attribute_index], i)
    return childs


# -----------------------------------------------------------------------

# function that modifies the dataset according to the impure childs
# -----------------------------------------------------------------------
def modify_data_set(dataset,index, feature, impurity):
    size = len(dataset)
    subdata = dataset[dataset[feature] == impurity]
    del (subdata[subdata.columns[index]])
    return subdata


# -----------------------------------------------------------------------


# function that return attribute with the greatest Information Gain
# -----------------------------------------------------------------------
def greatest_information_gain(dataset):
    max = -1
    attribute_index = 0
    size = len(dataset.columns) - 1
    for i in range(0, size):
        feature = list(dataset.iloc[:, i])
        i_g = Information_Gain(dataset, feature)
        if max < i_g:
            max = i_g
            attribute_index = i
    return attribute_index


# -----------------------------------------------------------------------


# function to construct the decision tree
# -----------------------------------------------------------------------
def construct_tree(dataset, tree):
    target = dataset.iloc[:, -1]
    impure_childs = []
    attribute_index = greatest_information_gain(dataset)
    childs = generate_childs(dataset, attribute_index)
    tree[dataset.columns[attribute_index]] = childs
    targets = list(set(dataset.iloc[:, -1]))
    for k, v in childs.items():
        if v[0] == 0:
            tree[k] = targets[1]
        elif v[1] == 0:
            tree[k] = targets[0]
        elif v[0] != 0 or v[1] != 0:
            impure_childs.append(k)
    for i in impure_childs:
        sub = modify_data_set(dataset,attribute_index, dataset.columns[attribute_index], i)
        tree = construct_tree(sub, tree)
    return tree


df = pd.read_csv(r"/content/7.bank.csv")
tree = dict()
result = construct_tree(df, tree)
for key, value in result.items():
    print(key, " => ", value)

balance  =>  {2343: (1, 0), 45: (5, 1), 1270: (4, 0), 2476: (1, 0), 184: (5, 2), 0: (662, 112), 830: (2, 2), 545: (3, 0), 1: (34, 5), 5090: (1, 0), 100: (10, 1), 309: (10, 1), 199: (7, 2), 460: (3, 0), 703: (9, 0), 3837: (1, 0), 611: (4, 0), -8: (3, 0), 55: (5, 1), 168: (10, 1), 785: (3, 0), 2067: (3, 0), 388: (11, 0), -192: (2, 0), 381: (6, 0), 40: (11, 1), 22: (16, 0), 3: (28, 7), 307: (11, 1), 759: (5, 0), -1: (8, 1), 65: (6, 2), 82: (6, 1), 10: (10, 1), 390: (11, 1), 311: (7, 1), 414: (9, 1), 5: (23, 4), 119: (8, 1), 4: (25, 4), 1262: (1, 1), 1949: (2, 0), -395: (2, 0), 1165: (8, 0), 2240: (6, 0), 300: (7, 1), 3285: (2, 0), 3923: (1, 0), 1443: (4, 0), 24: (11, 1), 1618: (1, 1), 517: (5, 4), 1521: (0, 2), 2823: (3, 0), 1405: (2, 1), 1535: (1, 0), 1596: (2, 0), 1542: (1, 0), 3652: (1, 0), 7180: (1, 0), 5291: (3, 0), 1384: (4, 0), -191: (2, 1), 320: (7, 2), 146: (8, 1), 341: (9, 3), -9: (2, 2), -306: (2, 2), 4580: (1, 0), 313: (11, 2), 10576: (1, 0), -233: (2, 0), 2453: (2, 0), 1364: 